In [125]:
import numpy as np
import sympy as sm
import sympy.physics.mechanics as me
import pydy

# Kinematics  

**Reference Frames:** Declare inertial reference frames and frames for each body

In [126]:
inertial_frame = me.ReferenceFrame('I')
lower_leg_frame = me.ReferenceFrame('L')
upper_leg_frame = me.ReferenceFrame('U')
torso_frame = me.ReferenceFrame('T')

**Coordinates:** Declaring coordinates for each of the joint angle.

In [127]:
theta1, theta2, theta3 = me.dynamicsymbols('theta1 theta2 theta3')

**Orient Frames:** Orient each of the reference frames according to the coordinate

In [128]:
lower_leg_frame.orient(inertial_frame,'Axis',(theta1,inertial_frame.z)) 
upper_leg_frame.orient(lower_leg_frame,'Axis',(theta2,lower_leg_frame.z))
torso_frame.orient(upper_leg_frame,'Axis',(theta3,upper_leg_frame.z))

**Joints:** Set the position of each of the joints

In [129]:
#Ankle
ankle = me.Point('A')
#Knee
lower_leg_length = sm.symbols('l_L')
knee = me.Point('K')
knee.set_pos(ankle,lower_leg_length*lower_leg_frame.y)
knee.pos_from(ankle)
#Hip
upper_leg_length = sm.symbols('l_U')
hip = me.Point('H')
hip.set_pos(knee,upper_leg_length*upper_leg_frame.y)
hip.pos_from(ankle)


l_L*L.y + l_U*U.y

**Centre of Mass:** Set the position of the CoM of each segment

In [130]:
#Lower leg
lower_leg_com_length = sm.symbols('d_L')
lower_leg_mass_center = me.Point('L_o')
lower_leg_mass_center.set_pos(ankle,lower_leg_com_length*lower_leg_frame.y)
lower_leg_mass_center.pos_from(ankle)

#Upper Leg
upper_leg_com_length = sm.symbols('d_U')
upper_leg_mass_center = me.Point('U_o')
upper_leg_mass_center.set_pos(knee,upper_leg_com_length*upper_leg_frame.y)
upper_leg_mass_center.pos_from(ankle)

#Torso
torso_com_length = sm.symbols('d_T')
torso_mass_center = me.Point('T_o')
torso_mass_center.set_pos(hip,torso_com_length*torso_frame.y)
torso_mass_center.pos_from(ankle)

l_L*L.y + l_U*U.y + d_T*T.y

**Kinematical Differential Equations:** Declare generalized speeds of each coordinate and relate them to the generalized coordinates to form the *Kinematical Differential Equations*

In [131]:
omega1, omega2, omega3 = me.dynamicsymbols('omega1 omega2 omega3')

kinematical_differential_equations = [omega1 - theta1.diff(),
                                      omega2 - theta2.diff(),
                                      omega3 - theta3.diff()]

**Angular Velcoities:** Now we can use the defined generalized speeds to specify the angular velocity of each of the coordinate system


In [132]:
#Lower Leg
lower_leg_frame.set_ang_vel(inertial_frame,omega1*inertial_frame.z)
lower_leg_frame.ang_vel_in(inertial_frame)

#Upper Leg
upper_leg_frame.set_ang_vel(lower_leg_frame,omega2*lower_leg_frame.z)
upper_leg_frame.ang_vel_in(inertial_frame)

#Torso
torso_frame.set_ang_vel(upper_leg_frame,omega3*upper_leg_frame.z)
torso_frame.ang_vel_in(inertial_frame)

omega1(t)*I.z + omega2(t)*L.z + omega3(t)*U.z

**Linear Velocities:** Define the linear velocity of each of the CoM starting from the ankle that has a linear velocity of zero

In [133]:
ankle.set_vel(inertial_frame,0)

#Lower Leg
lower_leg_mass_center.v2pt_theory(ankle,inertial_frame,lower_leg_frame)
lower_leg_mass_center.vel(inertial_frame)

#Knee
knee.v2pt_theory(ankle,inertial_frame,lower_leg_frame)
knee.vel(inertial_frame)

#Upper Leg
upper_leg_mass_center.v2pt_theory(knee,inertial_frame,upper_leg_frame)
upper_leg_mass_center.vel(inertial_frame)

#Hip
hip.v2pt_theory(knee,inertial_frame,upper_leg_frame)
hip.vel(inertial_frame)

#Torso
torso_mass_center.v2pt_theory(hip,inertial_frame,torso_frame)
torso_mass_center.vel(inertial_frame)


- l_L*omega1(t)*L.x - l_U*(omega1(t) + omega2(t))*U.x - d_T*(omega1(t) + omega2(t) + omega3(t))*T.x

# Inertia